In [167]:
import pandas as pd
import numpy as np

In [168]:
df = pd.read_csv(r"D:\Analysis Burn\Дистрибьютеры отчетность\Source\Отчеты\Москва OSHISHA\Sell_out_Москва_OSHISHA.csv")

In [169]:
df['Дата отгрузки'].str.match('\d{4}-.*').unique()

array([ True])

In [170]:
df['Дата отгрузки'] = df['Дата отгрузки'].str[:10]

In [171]:
df['Дата отгрузки'] = pd.to_datetime(df['Дата отгрузки'], yearfirst=True)

In [172]:
df['year'] = df['Дата отгрузки'].dt.year
df['month'] = df['Дата отгрузки'].dt.month
df['day'] = df['Дата отгрузки'].dt.day


In [173]:
df = df.loc[df['year'] == 2023]

In [174]:
df['year_month'] = df['year'].astype(str) + "_" + df['month'].astype(str)

In [175]:
df = df.pivot_table(
    index=['Торговый представитель', 'Код клиента',	'Сегмент', 'Клиент', 'Адрес доставки', 'year_month', 'Бренд', 'Группа', 'Вкус', 'Граммовка',],
    columns=['day'],
    values=['ВЕС'],
    aggfunc='sum'
).reset_index()

In [176]:
# Объединяем значения кортежей, пропуская 'Кг' и пустые значения
joined_values = ['_'.join([str(x) for x in df if x and x != pd.NaT]) for df in df.columns]

In [177]:
df.columns = pd.Series(joined_values).str.replace("ВЕС_", "")
df.columns = pd.Series(df.columns).str.replace("_NaT", "")

In [178]:
counts_in_data = len(df.iloc[:,10:].columns)

In [179]:
brands = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name="варианты брендов")

groups = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name="варианты групп")

delicious = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name="варианты вкусов")

gramming = pd.read_excel(r"D:\Analysis Burn\Продукция Burn.xlsx", sheet_name="варианты граммовок")

In [180]:
df = df.merge(
    brands,
    how="left",
    left_on="Бренд",
    right_on="Бренд (исходный)"
)

In [181]:
df['Бренд_y'].isna().sum()

0

In [182]:
df['Вкус'] = df['Бренд_y'] + df['Вкус']
df.loc[:, 'Вкус'] = df['Вкус'].str.upper()

In [183]:
df.rename(columns={'Вкус_x':'Вкус'}, inplace=True)

In [184]:
df = df.merge(
    delicious,
    how="left",
    left_on="Вкус",
    right_on="Исходный вкус"
)

In [185]:
df['Вкус_y'].isna().sum()

0

In [186]:
df_copy = df.copy(deep = True)

In [187]:
df.head(2)

,Торговый представитель,Код клиента,Сегмент,Клиент,Адрес доставки,year_month,Бренд_x,Группа,Вкус_x,Граммовка,...,27,28,29,30,31,Бренд (исходный),Бренд_y,Исходный вкус,Код,Вкус_y
0,kam 001 (шулагин),00-00000646,мопт,Чучин павел федорович,"Россия, 123458, москва г, вн.тер.г. муниципаль...",2023_1,B3,B3 50 гр,B3RICH GRAPE (БЕЛЫЙ ВИНОГРАД),50.0,...,NaN,NaN,NaN,NaN,NaN,B3,B3,B3RICH GRAPE (БЕЛЫЙ ВИНОГРАД),173,Rich Grape (Белый Виноград)
1,kam 001 (шулагин),00-00000646,мопт,Чучин павел федорович,"Россия, 123458, москва г, вн.тер.г. муниципаль...",2023_1,Banger,Banger 25 гр,"BANGERLAMBO (ДЫНЯ, МЕЛИССА, МЯТА)",25.0,...,NaN,NaN,NaN,NaN,NaN,Banger,Banger,"BANGERLAMBO (ДЫНЯ, МЕЛИССА, МЯТА)",102,"Lambo (Дыня, Мелисса, Мята)"


In [188]:
df = df.groupby(
    by=['year_month','Сегмент', 'Клиент', 'Адрес доставки' ,'Бренд_y', 'Вкус_y', 'Граммовка']
).sum(numeric_only=True).reset_index()

In [189]:
df.drop(columns=['Код'], inplace=True)

In [190]:
df['Кол-во закупок'] = df.iloc[:,7:].apply(np.count_nonzero, axis=1)

In [191]:
df['Кол-во закупок всего'] = counts_in_data

In [192]:
code_buy = df.groupby(
    by=['Клиент', 'year_month']    
).sum(numeric_only=True).drop(columns=['Кол-во закупок', 'Граммовка', 'Кол-во закупок всего']).reset_index()

In [193]:
code_buy['Кол-во закупок клиента'] = code_buy.iloc[:,2:].apply(np.count_nonzero, axis=1)

In [194]:
code_weight = df.groupby(
    by=['Клиент', 'year_month']    
).sum(numeric_only=True).drop(columns=['Кол-во закупок', 'Граммовка', 'Кол-во закупок всего']).reset_index()

code_weight['Вес закупок клиента'] = code_weight.iloc[:,2:].sum(axis=1)

code_weight = code_weight.loc[:, ['Клиент','year_month', 'Вес закупок клиента']]

code_weight

,Клиент,year_month,Вес закупок клиента
0,andrew's tobacco (нововладыкинский 1к3),2023_7,3.325
1,andrew's tobacco (нововладыкинский 1к3),2023_8,0.100
2,coco lounge (просторная ),2023_1,1.100
3,coco lounge (просторная ),2023_2,3.400
4,coco lounge (просторная ),2023_6,1.000
...,...,...,...
16844,Ясин 3-39 юв,2023_5,9.325
16845,Ястребов андрей дмитриевич,2023_8,1.000
16846,Ященко дмитрий александрович,2023_5,0.200
16847,Ященко дмитрий александрович,2023_4,0.300


In [195]:
final_table = df.merge(
    code_buy.loc[:, ['Клиент', 'year_month', 'Кол-во закупок клиента']],
    on=['Клиент', 'year_month']
)

In [196]:
final_table['Объем_клиента'] = final_table.iloc[:,7:-3].sum(axis=1)

In [197]:
final_table['Ср. объем_клиента'] = final_table['Объем_клиента'] / final_table['Кол-во закупок']

In [198]:
final_table = final_table.merge(
    code_weight.loc[:, ['Клиент', 'year_month', 'Вес закупок клиента']],
    on=['Клиент', 'year_month']
)

In [199]:
final_table.columns

Index(['year_month', 'Сегмент', 'Клиент', 'Адрес доставки', 'Бренд_y',
       'Вкус_y', 'Граммовка', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',
       'Кол-во закупок', 'Кол-во закупок всего', 'Кол-во закупок клиента',
       'Объем_клиента', 'Ср. объем_клиента', 'Вес закупок клиента'],
      dtype='object')

In [200]:
final_table = final_table[['year_month', 'Сегмент', 'Клиент', 'Адрес доставки', 'Бренд_y', 'Вкус_y', 'Граммовка', 'Кол-во закупок',
'Кол-во закупок клиента', 'Объем_клиента', 'Ср. объем_клиента','Вес закупок клиента']]

In [201]:
final_table['Доля вкуса во всех закупках'] = final_table['Кол-во закупок'] / final_table['Кол-во закупок клиента'] * 100

final_table['Доля вкуса во всём объеме закупок закупках'] = final_table['Объем_клиента'] / final_table['Вес закупок клиента'] * 100

In [202]:
final_table.rename(columns={
        'Бренд_y': 'Бренд', 
        'Вкус_y' : 'Вкус', 
        'Кол-во закупок' : 'Кол-во закупок вкуса', 
        'Объем_клиента':'Общий объем вкуса', 
        'Ср. объем_клиента':'Ср. объем клиента', 
        'Вес закупок клиента':'Объем закупок клиента',
        'Доля вкуса во всём объеме закупок закупках':'Доля вкуса во всём объеме закупок'
    }, inplace=True
)

In [203]:
final_table['Доля вкуса во всех закупках'] = final_table['Доля вкуса во всех закупках'] / 100
final_table['Доля вкуса во всём объеме закупок'] = final_table['Доля вкуса во всём объеме закупок'] / 100

In [102]:
final_table.to_excel(r"D:\Analysis Burn\SKU_Анализ\data\simple_SKU_full.xlsx")